In [142]:
import pandas as pd
import datetime

In [44]:
df = pd.read_csv('data/completo.csv', delimiter=";")

/tmp/ipykernel_23095/2374307776.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/completo.csv', delimiter=";")


In [85]:
dataset = [[m//3 + 1, 0] for m in range(12*3)]

for i in range(len(dataset)):
    if i % 3 == 0:
        dataset[i].append("App")
    elif i % 3 == 1:
        dataset[i].append("Web")   
    else:
        dataset[i].append("Otros")



for row in df.values:
    if type(row[4]) == str:
        fecha = datetime.datetime.strptime(row[4], "%d/%m/%Y")
        if row[12] == "App BA 147" or row[12] == "App Denuncia Vial":
            dataset[(fecha.month-1)*3][1] += 1
        elif row[12] == "GCS Web":
            dataset[(fecha.month-1)*3+1][1] += 1
        else:
            dataset[(fecha.month-1)*3+2][1] += 1

In [86]:
df_red = pd.DataFrame(dataset)
df_red.columns = ["Mes", "Valor", "Canal"]

In [88]:
df_red.to_csv("denuncias_por_mes.csv")

In [171]:
# dataset = [Barrio, Denuncias Julio, Denuncias Agosto]
barrios = df["domicilio_barrio"].unique().tolist()[1:]
dataset = pd.DataFrame()
dataset["Barrio"] = barrios
dataset["Julio"] = [0 for _ in range(len(barrios))]
dataset["Agosto"] = [0 for _ in range(len(barrios))]

for row in df.values:
    if type(row[4]) == str:
        fecha = datetime.datetime.strptime(row[4], "%d/%m/%Y")
        if not pd.isnull(row[7]):
            if fecha.month == 4:
                dataset.loc[dataset['Barrio'] == row[7], "Julio"] += 1
            elif fecha.month == 8:
                dataset.loc[dataset['Barrio'] == row[7], "Agosto"] += 1


In [172]:
dataset.to_csv("denuncias_por_barrio.csv")